# `auto_arima`

Pyramid bring R's [`auto.arima`](https://www.rdocumentation.org/packages/forecast/versions/7.3/topics/auto.arima) functionality to Python by wrapping statsmodel [`ARIMA`](https://github.com/statsmodels/statsmodels/blob/master/statsmodels/tsa/arima_model.py) and [`SARIMAX`](https://github.com/statsmodels/statsmodels/blob/master/statsmodels/tsa/statespace/sarimax.py) models into a singular scikit-learn-esque estimator ([`pyramid.arima.ARIMA`](https://github.com/tgsmith61591/pyramid/blob/master/pyramid/arima/arima.py)) and adding several layers of degree and seasonal differencing tests to identify the optimal model parameters.

__Pyramid ARIMA models:__

  - Are fully picklable for easy persistence and model deployment
  - Can handle seasonal terms (unlike statsmodels ARIMAs)
  - Follow sklearn model fit/predict conventions

In [1]:
import numpy as np
import pyramid

print('numpy version: %r' % np.__version__)
print('pyramid version: %r' % pyramid.__version__)

numpy version: '1.15.0'
pyramid version: '0.6.5'


In [2]:
from bokeh.plotting import figure, show, output_notebook
import pandas as pd

# init bokeh
output_notebook()

def plot_arima(truth, forecasts, title="ARIMA", xaxis_label='Time',
               yaxis_label='Value', c1='#A6CEE3', c2='#B2DF8A', 
               forecast_start=None, **kwargs):
    
    # make truth and forecasts into pandas series
    n_truth = truth.shape[0]
    n_forecasts = forecasts.shape[0]
    
    # always plot truth the same
    truth = pd.Series(truth, index=np.arange(truth.shape[0]))
    
    # if no defined forecast start, start at the end
    if forecast_start is None:
        idx = np.arange(n_truth, n_truth + n_forecasts)
    else:
        idx = np.arange(forecast_start, n_forecasts)
    forecasts = pd.Series(forecasts, index=idx)
    
    # set up the plot
    p = figure(title=title, plot_height=400, **kwargs)
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = yaxis_label
    
    # add the lines
    p.line(truth.index, truth.values, color=c1, legend='Observed')
    p.line(forecasts.index, forecasts.values, color=c2, legend='Forecasted')
    
    return p

/home/jon/.pyenv/versions/3.6.4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jon/.pyenv/versions/3.6.4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Loading BokehJS ...

In [3]:
import pandas as pd
wineind=pd.read_csv("/home/jon/h2oai/tests/data/train_sine.csv", header=None).values[:, 1]
wineind = wineind - np.average(wineind)
print(wineind)
np.average(wineind)

[-2.14959598 -2.14776755 -2.14539982 ... -7.55554444 -7.38594522
 -7.21425239]


-9.094947017729283e-16

## Fitting an ARIMA

Note that your data does not have to exhibit seasonality to work with an ARIMA. We could fit an ARIMA against the same data with no seasonal terms whatsoever (but it is unlikely that it will perform better; quite the opposite, likely).

In [4]:
from pyramid.arima import ARIMA
fit = ARIMA(order=(4,0,1)).fit(y=wineind)

/home/jon/.pyenv/versions/3.6.4/lib/python3.6/site-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [5]:
in_sample_preds = fit.predict_in_sample()
in_sample_preds[:10]

array([-5.10639537e-03, -2.36093052e+01, -6.23009135e+00, -4.31896335e+00,
       -3.58140277e+00, -3.18996871e+00, -2.94746922e+00, -2.78263773e+00,
       -2.66336427e+00, -2.57302069e+00])

In [6]:
show(plot_arima(wineind, in_sample_preds, 
                title="Original Series & In-sample Predictions", 
                c2='#FF0000', forecast_start=0))

In [7]:
next_5000 = fit.predict(n_periods=5000)
next_5000
# call the plotting func
show(plot_arima(wineind, next_5000))

/home/jon/.pyenv/versions/3.6.4/lib/python3.6/site-packages/statsmodels/tsa/arima_model.py:1514: RuntimeWarning: invalid value encountered in sqrt
  fcasterr = np.sqrt(sigma2 * np.cumsum(ma_rep**2))


## Finding the optimal model hyper-parameters using `auto_arima`:

If you are unsure (as is common) of the best parameters for your model, let `auto_arima` figure it out for you. `auto_arima` is similar to an ARIMA-specific grid search, but (by default) uses a more intelligent `stepwise` algorithm laid out in a paper by Hyndman and Khandakar (2008). If `stepwise` is False, the models will be fit similar to a gridsearch. Note that it is possible for `auto_arima` not to find a model that will converge; if this is the case, it will raise a `ValueError`.

`auto_arima` can fit a random search that is much faster than the exhaustive one by enabling `random=True`. If your random search returns too many invalid (nan) models, you might try increasing `n_fits` or making it an exhaustive search.

In [8]:
# fitting a stepwise model:
from pyramid.arima import auto_arima

stepwise_fit = auto_arima(wineind, start_p=1, start_q=1, max_p=4, max_q=4, m=12,
                          start_P=0, seasonal=True, d=0, D=1, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True, nfits=100)  # set to stepwise

stepwise_fit.summary()

# in-sample
in_sample_preds = stepwise_fit.predict_in_sample()
in_sample_preds[:10]
show(plot_arima(wineind, in_sample_preds, 
                title="stepwise: Original Series & In-sample Predictions", 
                c2='#FF0000', forecast_start=0))
# forecasting
next_5000 = stepwise_fit.predict(n_periods=5000)
next_5000
# call the plotting func
show(plot_arima(wineind, next_5000))

Fit ARIMA: order=(1, 0, 1) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 1, 0, 12); AIC=31860.050, BIC=31874.468, Fit time=0.626 seconds
Fit ARIMA: order=(1, 0, 0) seasonal_order=(1, 1, 0, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 0, 1) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(1, 1, 0, 12); AIC=-3458.486, BIC=-3436.859, Fit time=31.032 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(1, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(2, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 0, 1) seasonal_order=(1, 1, 0, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 0, 1) seasonal_order=(1, 1, 0, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(2, 1, 0, 12); AIC=-50168.043, BIC=-50139.206, Fit time=69.252 se

In [9]:
rs_fit = auto_arima(wineind, start_p=1, start_q=1, max_p=4, max_q=3, m=12,
                    start_P=0, seasonal=False, n_jobs=-1, d=0, D=1, trace=True,
                    error_action='ignore',  # don't want to know if an order does not work
                    suppress_warnings=True,  # don't want convergence warnings
                    stepwise=False, random=True, random_state=42,  # we can fit a random search (not exhaustive)
                    n_fits=25)

rs_fit.summary()

# in-sample
in_sample_preds = rs_fit.predict_in_sample()
in_sample_preds[:10]
show(plot_arima(wineind, in_sample_preds, 
                title="rs: Original Series & In-sample Predictions", 
                c2='#FF0000', forecast_start=0))
# forecasting
next_5000 = rs_fit.predict(n_periods=5000)
next_5000
# call the plotting func
show(plot_arima(wineind, next_5000))

Fit ARIMA: order=(4, 0, 2); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(2, 0, 2); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(3, 0, 1); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(3, 0, 2); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(4, 0, 3); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 0, 1); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(2, 0, 3); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 0, 3); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 0, 2); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(3, 0, 3); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(2, 0, 1); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(4, 0, 1); AIC=nan, BIC=nan, Fit time=16.743 seconds


/home/jon/.pyenv/versions/3.6.4/lib/python3.6/site-packages/statsmodels/tsa/arima_model.py:1630: RuntimeWarning: invalid value encountered in double_scalars
  ('S.D. of innovations', ["%#5.3f" % self.sigma2**.5]),
/home/jon/.pyenv/versions/3.6.4/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/jon/.pyenv/versions/3.6.4/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/jon/.pyenv/versions/3.6.4/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


Total fit time: 18.916 seconds


/home/jon/.pyenv/versions/3.6.4/lib/python3.6/site-packages/statsmodels/tsa/arima_model.py:1630: RuntimeWarning: invalid value encountered in double_scalars
  ('S.D. of innovations', ["%#5.3f" % self.sigma2**.5]),


/home/jon/.pyenv/versions/3.6.4/lib/python3.6/site-packages/statsmodels/tsa/arima_model.py:1514: RuntimeWarning: invalid value encountered in sqrt
  fcasterr = np.sqrt(sigma2 * np.cumsum(ma_rep**2))


## Updating your model

ARIMAs create forecasts by using the latest observations. Over time, your forecasts will drift, and you'll need to update the model with the observed values. The _current_ solution is to re-fit the ARIMA obtained from `auto_arima` with the new data. This way, the order (e.g., p, d, q) and other args stay the same. For this example, let us add in the forecasted values, as if they were actual observed values.

In [11]:
updated_data = np.concatenate([wineind, next_5000])
updated_model = stepwise_fit.fit(updated_data)
updated_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            Statespace Model Results                            
================================================================================
Dep. Variable:                        y   No. Observations:                15000
Model:             SARIMAX(2, 1, 0, 12)   Log Likelihood               21709.991
Date:                  Wed, 25 Jul 2018   AIC                         -43411.982
Time:                          22:51:18   BIC                         -43381.522
Sample:                               0   HQIC                        -43401.875
                                - 15000                                         
Covariance Type:                    opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept  -2.062e-05      0.000     -0.042      0.967      -0.001       0.001
ar.S.L12       1.9641      0.000   8631.669      0.000       1.964       1.965
ar.S.L24      -0.9992      0.000  -4354.577      0.000      -1.000      -0.999
sigma2         0.0032   1.14e-05    279.998      0.000       0.003       0.003
===================================================================================
Ljung-Box (Q):                   546104.24   Jarque-Bera (JB):            283542.57
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):              15.81   Skew:                            -1.34
Prob(H) (two-sided):                  0.00   Kurtosis:                        24.14
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [12]:
# visualize new forecasts
show(plot_arima(updated_data, updated_model.predict(n_periods=5000)))